Preprocessing

In [162]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

# Build path to file
# Selects current working directory
cwd = Path.cwd()
csv_path = 'data/dataframe.csv'
csv_cleaned_path = 'output/dataframe_cleaned.csv'
src_path = (cwd / csv_path).resolve()
out_path = (cwd / csv_cleaned_path).resolve()

# Read the csv
csv = pd.read_csv(src_path, index_col=0)
csv.shape

# Removing unneeded data
# dropping empty rows
csv = csv.dropna(how='all')
# dropping duplicates (if any)
csv = csv.drop_duplicates()
# Dropping house and appartment groups since they have no data
csv = csv.drop(csv[(csv['property_type'] == 'HOUSE_GROUP') | (csv['property_type'] == 'APARTMENT_GROUP')].index)
# Drop rows without a price property
csv = csv.drop(csv[pd.isna(csv['price']) == True].index)
# Drop rows with 0 rooms
csv = csv.drop(csv[csv['number_rooms'] == 0].index)
# Drop rows without a living area property
csv = csv.drop(csv[pd.isna(csv['living_area']) == True].index)
# Assuming that a NaN value or 0 means no kitchen installed, replacing the strings with integers
# 0 = NOT_INSTALLED, 0.5 = SEMI_EQUIPPED, 1 = INSTALLED, 2 = HYPER_EQUIPPED
csv['kitchen'] = csv['kitchen'].fillna('NOT_INSTALLED')
csv['kitchen'] = csv['kitchen'].replace('0', 'NOT_INSTALLED')
csv['kitchen'] = csv['kitchen'].replace(0, 'NOT_INSTALLED')
"""csv['kitchen'] = csv['kitchen'].replace('USA_UNINSTALLED', 0)
csv['kitchen'] = csv['kitchen'].replace('SEMI_EQUIPPED', 0.5)
csv['kitchen'] = csv['kitchen'].replace('USA_SEMI_EQUIPPED', 0.5)
csv['kitchen'] = csv['kitchen'].replace('INSTALLED', 1)
csv['kitchen'] = csv['kitchen'].replace('USA_INSTALLED', 1)
csv['kitchen'] = csv['kitchen'].replace('HYPER_EQUIPPED', 2)
csv['kitchen'] = csv['kitchen'].replace('USA_HYPER_EQUIPPED', 2)"""
# Filling empty values and changing true/false to 1/0
csv['furnished'] = csv['furnished'].fillna(0)
csv['furnished'] = csv['furnished'].replace(False, 0)
csv['furnished'] = csv['furnished'].replace(True, 1)
# Assuming that a NaN value,0 or -1 means no fireplace installed
csv['fireplace'] = csv['fireplace'].fillna(0)
csv['fireplace'] = csv['fireplace'].replace(-1, 0)
# Filling empty values and changing true/false to 1/0
csv['terrace'] = csv['terrace'].fillna(0)
csv['terrace'] = csv['terrace'].replace(False, 0)
csv['terrace'] = csv['terrace'].replace(True, 1)
# Assuming the surface area = living area in case of apartments
to_replace = csv[((csv['surface_land'] == 'UNKNOWN')|(pd.isna(csv['surface_land']) == True)) & (csv['property_type'] == 'APARTMENT')]
to_replace = to_replace.reset_index()
# Looping through rows to replace the values
for index, row in to_replace.iterrows():
    csv.loc[row['index'], 'surface_land'] = row['living_area']
# Dropping rows with no surface area 
csv = csv.drop(csv[(csv['surface_land'] == 'UNKNOWN') | (pd.isna(csv['surface_land']) == True) | (csv['surface_land'] == 0)].index)
# Dropping rows with no facade info
csv = csv.drop(csv[(csv['number_facades'] == 'UNKNOWN') | (pd.isna(csv['number_facades']) == True)].index)
# Filling empty values and changing true/false to 1/0
csv['swimming_pool'] = csv['swimming_pool'].fillna(0)
csv['swimming_pool'] = csv['swimming_pool'].replace(False, 0)
csv['swimming_pool'] = csv['swimming_pool'].replace(True, 1)
csv = csv.drop(csv[(csv['building_state'] == 'UNKNOWN') | (pd.isna(csv['building_state']) == True)].index)

# If terrace = 1 but no terrace_area present, drop the row
csv = csv.drop(csv[(csv['terrace'] == 1) & (pd.isna(csv['terrace_area']) == True)].index)
# Filling empty values and changing true/false to 1/0
csv['terrace_area'] = csv['terrace_area'].fillna(0)
# If garden = 1 but no garden_area present, drop the row
csv = csv.drop(csv[(csv['garden'] == 1) & (pd.isna(csv['garden_area']) == True)].index)
# No garden, filling empty values
csv['garden'] = csv['garden'].fillna(0)
csv['garden'] = csv['garden'].replace(False, 0)
csv['garden'] = csv['garden'].replace(True, 1)
csv['garden_area'] = csv['garden_area'].fillna(0)

# Change strings to floats in certain columns
csv = csv.drop(csv[(csv['surface_land'] == 0)].index)
csv['surface_land']=csv['surface_land'].astype("float")
csv['number_facades']=csv['number_facades'].astype("float")
csv = csv.drop(csv[csv['zip_code'] == 'UNKNOWN'].index)
csv = csv.drop(csv[pd.isna(csv['region']) == True].index)
csv = csv.drop(csv[pd.isna(csv['province']) == True].index)
csv['zip_code']=csv['zip_code'].astype("str")
csv['ppm'] = csv['price']/csv['surface_land']
# Removing zipcodes that are not 4 numbers
patternDel = "\b[0-9]\{4\}\b"
filter = csv['zip_code'].str.contains(patternDel)
csv = csv[~filter]
# If we have less than 3 occurences, zipcode will be changed to 'other' so we don't overfit
filter = csv['zip_code'].value_counts()
csv['zip_code'] = np.where(csv['zip_code'].isin(filter.index[filter >= 4]), csv['zip_code'], 'other')
# Removing outliers
cols = ['price', 'number_rooms', 'living_area',
       'furnished', 'fireplace', 'terrace', 'terrace_area', 'garden',
       'garden_area', 'surface_land', 'number_facades', 'swimming_pool'] # one or more
Q1 = csv[cols].quantile(0.25)
Q3 = csv[cols].quantile(0.75)
IQR = Q3 - Q1
csv = csv[~((csv[cols] < (Q1 - 1.5 * IQR)) |(csv[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

# Saves cleaned up csv to 'data/dataframe_cleaned.csv'
csv.to_csv(out_path)
csv.shape


(4774, 22)

In [163]:
def convert(n):
    if n == 'other':
        return 'other'
    else:
        return str(int(int(n)/100))
csv["digit"]=csv["zip_code"].agg(convert)

Linear Regression

In [307]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score

x = csv[['number_rooms', 'living_area',
       'terrace', 'terrace_area', 'garden',
       'garden_area', 'surface_land', 'number_facades',
       'property_type', 'building_state', 'kitchen', 'region', 'digit']]

x = pd.get_dummies(data=x, drop_first=True)
X = x.to_numpy()
y = csv['price'].to_numpy()

print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(4774, 98)


In [230]:
def score (regressor, X_train, X_test, y_train, y_test):
    score_train = regressor.score(X_train, y_train)
    score_test = regressor.score(X_test, y_test)

    # Get the root mean squared error
    y_pred = regressor.predict(X_test)
    #rmse = mean_squared_error(y_true=y_test, y_pred=y_pred, squared=False)
    rmse= np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred))

    u = ((y_test - y_pred)**2).sum()
    v = ((y_test - y.mean())**2).sum()
    coef_determination = 1 - u/v

    print(f"score train:{score_train}")
    print(f"score test:{score_test}")
    print(f"rmse:{rmse}")
    print(f"coef_determination:{coef_determination}")

In [308]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

score(regressor, X_train, X_test, y_train, y_test)

score train:0.7200823716194424
score test:0.6880006377098221
rmse:94674.7084296804
coef_determination:0.6880574910104393


In [ ]:
# TODO: zijn er variabelen met extreme coefficienten?
# TODO: welke zijn de hoogste predictions, hoe zien de X eruit daarvoor?
# TODO: normaliseer numerical variabelen tussen 0-1 (StandardScaler())
# TODO: zipcode minder granulair?
dict_coef = {x.columns[i]: regressor.coef_[i] for i in range(len(x.columns))}
dict_coef

In [232]:
"""np.quantile(y_pred, q=0.90)
pd.DataFrame(y_pred).hist(bins=10)"""

734689.0855482789

In [314]:
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import decomposition
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
"""
std_slc = StandardScaler()
pca = decomposition.PCA()
dec_tree = DecisionTreeClassifier()

pipe = Pipeline(steps=[('std_slc', std_slc),
                           ('pca', pca),
                           ('dec_tree', dec_tree)])

n_components = list(range(1,X_train.shape[1]+1,1))
criterion = ['gini', 'entropy']
max_depth = [2,4,6,8,10,12]

parameters = dict(pca__n_components=n_components,
                      dec_tree__criterion=criterion,
                      dec_tree__max_depth=max_depth)

clf_GS = GridSearchCV(pipe, parameters)
clf_GS.fit(X_train, y_train)

print('Best Criterion:', clf_GS.best_estimator_.get_params()['dec_tree__criterion'])
print('Best max_depth:', clf_GS.best_estimator_.get_params()['dec_tree__max_depth'])
print('Best Number Of Components:', clf_GS.best_estimator_.get_params()['pca__n_components'])
params = clf_GS.best_estimator_.get_params()['dec_tree']
print(); print(params)"""

c:\Users\Ridd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best Criterion: entropy
Best max_depth: 12
Best Number Of Components: 98

DecisionTreeClassifier(criterion='entropy', max_depth=12)


In [331]:
regressor = DecisionTreeClassifier(ccp_alpha=0.001, random_state=0)
regressor.fit(X_train, y_train).tree_.node_count

score(regressor, X_train, X_test, y_train, y_test)
cross_val_score(regressor, X_test, y_test, cv = 5, scoring = 'accuracy').mean()

score train:0.027094972067039105
score test:0.031825795644891124
rmse:135263.57145613714
coef_determination:0.36325094551473147


c:\Users\Ridd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.03852185225554657

In [333]:
regressor = DecisionTreeClassifier(criterion='entropy', max_depth=12)
regressor.fit(X_train, y_train)

score(regressor, X_train, X_test, y_train, y_test)
cross_val_score(regressor, X_test, y_test, cv = 5, scoring = 'accuracy').mean()

score train:0.8360335195530726
score test:0.09045226130653267
rmse:142332.29749213735
coef_determination:0.294960372451204


c:\Users\Ridd\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.03852536830631834

In [310]:
import xgboost as xg

regressor = xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 50, seed = 123)
regressor.fit(X_train, y_train)

score(regressor, X_train, X_test, y_train, y_test)

score train:0.8955140520285985
score test:0.7231510458472554
rmse:89182.27909100034
coef_determination:0.7232014939529838


In [312]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline

regressor = SGDRegressor(max_iter=1000, tol=1e-3)
regressor.fit(X_train, y_train)

score(regressor, X_train, X_test, y_train, y_test)

score train:0.7172414777500633
score test:0.6801913395316415
rmse:95852.23267181792
coef_determination:0.6802496158620164


In [293]:
from sklearn.neural_network import MLPClassifier

regressor = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1, max_iter=1000)
regressor.fit(X_train, y_train)

score(regressor, X_train, X_test, y_train, y_test)

score train:0.0611731843575419
score test:0.038525963149078725
rmse:108523.94056082008
coef_determination:0.5978783265395025
